In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of huggingface-hub[torch] to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of datasets to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking 

## Sıfır Aktivite Kayıtları Silmek

In [2]:
import pandas as pd

try:
    # Müşteri işlem geçmişi dosyasını yükleyelim
    customer_history = pd.read_csv('/kaggle/input/ing-hubs-turkiye-datathon/customer_history.csv')

    print(f"Temizlemeden önceki toplam kayıt sayısı: {len(customer_history)}")

    # İşlem yapılıp yapılmadığını anlamak için aktivite sütunlarını belirleyelim
    activity_cols = [
        'mobile_eft_all_cnt', 'mobile_eft_all_amt',
        'cc_transaction_all_cnt', 'cc_transaction_all_amt'
    ]

    # Bu sütunların hepsinin sıfır olduğu satırları "sıfır aktivite" olarak kabul edelim
    is_zero_activity = (customer_history[activity_cols] == 0).all(axis=1)

    # Sadece aktivitesi olan (en az bir işlem tutarı veya adedi sıfırdan büyük olan) satırları tutalım
    cleaned_customer_history = customer_history[~is_zero_activity].copy()

    print(f"Temizledikten sonraki gerçek işlem sayısı: {len(cleaned_customer_history)}")
    print(f"Kaldırılan sıfır aktivite kaydı sayısı: {len(customer_history) - len(cleaned_customer_history)}")

    print("\nSONUÇ: Veri seti artık sadece gerçek müşteri işlemlerini içeriyor.")

    # ÖNEMLİ: Bundan sonraki tüm adımlarda (create_features vb.)
    # orijinal 'customer_history' yerine 'cleaned_customer_history' DataFrame'ini kullanmalısınız.

except FileNotFoundError:
    print("Hata: 'customer_history.csv' dosyası bulunamadı.")

Temizlemeden önceki toplam kayıt sayısı: 5359609
Temizledikten sonraki gerçek işlem sayısı: 5311049
Kaldırılan sıfır aktivite kaydı sayısı: 48560

SONUÇ: Veri seti artık sadece gerçek müşteri işlemlerini içeriyor.


## Feature Engineering

In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from autogluon.tabular import TabularPredictor
from sklearn.metrics import roc_auc_score
import warnings
warnings.filterwarnings('ignore')

customer_history = pd.read_csv('/kaggle/input/ing-hubs-turkiye-datathon/customer_history.csv')
customers = pd.read_csv('/kaggle/input/ing-hubs-turkiye-datathon/customers.csv')
referance_data = pd.read_csv('/kaggle/input/ing-hubs-turkiye-datathon/referance_data.csv')
referance_data_test = pd.read_csv('/kaggle/input/ing-hubs-turkiye-datathon/referance_data_test.csv')
activity_cols = ['mobile_eft_all_cnt', 'mobile_eft_all_amt', 'cc_transaction_all_cnt', 'cc_transaction_all_amt']
is_zero_activity = (customer_history[activity_cols] == 0).all(axis=1)
customer_history = customer_history[~is_zero_activity].copy()

def create_features(customer_history, customers, reference_data, is_train=True, full_train_df_for_thresholds=None):
    print("Kanıtlanmış özellik seti oluşturuluyor...")

    customer_history['date'] = pd.to_datetime(customer_history['date'])
    reference_data['ref_date'] = pd.to_datetime(reference_data['ref_date'])
    customer_history['month_period'] = customer_history['date'].dt.to_period('M')

    df = customer_history.merge(reference_data, on='cust_id', how='inner')
    df = df[df['date'] < df['ref_date']].copy()
    df['days_before_ref'] = (df['ref_date'] - df['date']).dt.days

    # Initialize final_features by merging reference_data and customers
    final_features = reference_data[['cust_id', 'ref_date']].copy()
    if is_train:
        final_features = final_features.merge(reference_data[['cust_id', 'churn']], on='cust_id', how='left')
    final_features = final_features.merge(customers, on='cust_id', how='left')


    # YENİ ÖZELLİKLERİ EKLE
    # 1. Tenure Gruplama
    bins = [-1, 90, 180, 270, float('inf')]
    labels = ['Çok Yeni (0-3 Ay)', 'Yeni (3-6 Ay)', 'Yerleşik (6-9 Ay)', 'Kıdemli (9+ Ay)']
    # Ensure 'tenure' column is present before creating 'tenure_group'
    if 'tenure' in final_features.columns:
      final_features['tenure_group'] = pd.cut(final_features['tenure'], bins=bins, labels=labels, right=False)
    else:
      final_features['tenure_group'] = 'Unknown' # Handle cases where 'tenure' is missing


    # Calculate total spending and merge it to final_features
    total_spending_df = df.groupby('cust_id')[['mobile_eft_all_amt', 'cc_transaction_all_amt']].sum().sum(axis=1).reset_index(name='total_amt')
    final_features = final_features.merge(total_spending_df, on='cust_id', how='left')


    if is_train:
        high_spender_threshold = final_features['total_amt'].quantile(0.99)
    else:
        # Use the threshold calculated from the full training data
        high_spender_threshold = full_train_df_for_thresholds['total_amt'].quantile(0.99)

    # Ensure 'total_amt' is present before creating 'is_high_spender'
    if 'total_amt' in final_features.columns:
      final_features['is_high_spender'] = (final_features['total_amt'] >= high_spender_threshold).astype(int)
    else:
      final_features['is_high_spender'] = 0 # Default to 0 if 'total_amt' is missing


    if is_train:
        # Ensure 'work_type' and 'total_amt' are present before calculating avg_amt_by_work_type
        if 'work_type' in final_features.columns and 'total_amt' in final_features.columns:
          avg_amt_by_work_type = final_features.groupby('work_type')['total_amt'].transform('mean')
        else:
          avg_amt_by_work_type = 0 # Default to 0 if necessary columns are missing
    else:
        # Use the average amounts calculated from the full training data
        if 'work_type' in final_features.columns and 'total_amt' in full_train_df_for_thresholds.columns:
          avg_amt_map = full_train_df_for_thresholds.groupby('work_type')['total_amt'].mean()
          avg_amt_by_work_type = final_features['work_type'].map(avg_amt_map).fillna(0)
        else:
          avg_amt_by_work_type = 0 # Default to 0 if necessary columns are missing

    # Ensure 'total_amt' and 'amt_vs_work_type_avg_ratio' are present before calculation
    if 'total_amt' in final_features.columns:
      final_features['amt_vs_work_type_avg_ratio'] = final_features['total_amt'] / (avg_amt_by_work_type + 1e-6)
    else:
      final_features['amt_vs_work_type_avg_ratio'] = 0 # Default to 0 if 'total_amt' is missing


    features_list = []
    for period in [30, 60, 90, 180, 365]:
        temp = df[df['days_before_ref'] <= period].copy()
        active_months = temp.groupby('cust_id')['month_period'].nunique().reset_index().rename(columns={'month_period': f'active_months_last_{period}d'})

        agg_features = temp.groupby('cust_id').agg({'mobile_eft_all_cnt': ['sum', 'mean', 'std', 'max'],'mobile_eft_all_amt': ['sum', 'mean', 'std', 'max'],'cc_transaction_all_cnt': ['sum', 'mean', 'std', 'max'],'cc_transaction_all_amt': ['sum', 'mean', 'std', 'max'],'active_product_category_nbr': ['mean', 'max', 'min'],'date': 'count'}).reset_index()
        agg_features.columns = ['cust_id'] + [f'{col[0]}_{col[1]}_last_{period}d' for col in agg_features.columns[1:]]

        # YENİ: İşlem Başına Ortalama Tutar
        agg_features[f'mobile_amt_per_txn_last_{period}d'] = agg_features[f'mobile_eft_all_amt_sum_last_{period}d'] / (agg_features[f'mobile_eft_all_cnt_sum_last_{period}d'] + 1e-6)
        agg_features[f'cc_amt_per_txn_last_{period}d'] = agg_features[f'cc_transaction_all_amt_sum_last_{period}d'] / (agg_features[f'cc_transaction_all_cnt_sum_last_{period}d'] + 1e-6)


        agg_features = agg_features.merge(active_months, on='cust_id', how='left')
        agg_features[f'zero_activity_months_last_{period}d'] = (period/30) - agg_features[f'active_months_last_{period}d']

        features_list.append(agg_features)

    all_time_agg = df.groupby('cust_id').agg({'mobile_eft_all_cnt': ['sum', 'mean', 'std', 'max'],'mobile_eft_all_amt': ['sum', 'mean', 'std', 'max'],'cc_transaction_all_cnt': ['sum', 'mean', 'std', 'max'],'cc_transaction_all_amt': ['sum', 'mean', 'std', 'max'],'active_product_category_nbr': ['mean', 'max', 'min'],'date': 'count'}).reset_index()
    all_time_agg.columns = ['cust_id'] + [f'{col[0]}_{col[1]}_all_time' for col in all_time_agg.columns[1:]]

    features_list.append(all_time_agg)

    trend_features = pd.DataFrame({'cust_id': reference_data['cust_id'].unique()})

    last_30 = df[df['days_before_ref'] <= 30].groupby('cust_id').agg({'mobile_eft_all_amt': 'sum', 'cc_transaction_all_amt': 'sum'}).reset_index()
    last_30.columns = ['cust_id', 'mobile_amt_30d', 'cc_amt_30d']
    last_60 = df[df['days_before_ref'] <= 60].groupby('cust_id').agg({'mobile_eft_all_amt': 'sum', 'cc_transaction_all_amt': 'sum'}).reset_index()
    last_60.columns = ['cust_id', 'mobile_amt_60d', 'cc_amt_60d']

    trend_features = trend_features.merge(last_30, on='cust_id', how='left').merge(last_60, on='cust_id', how='left')
    trend_features['mobile_trend'] = trend_features['mobile_amt_30d'] / (trend_features['mobile_amt_60d'] + 1)
    trend_features['cc_trend'] = trend_features['cc_amt_30d'] / (trend_features['cc_amt_60d'] + 1)

    features_list.append(trend_features[['cust_id', 'mobile_trend', 'cc_trend']])

    last_transaction = df.groupby('cust_id').agg({'date': 'max', 'ref_date': 'first'}).reset_index()
    last_transaction['days_since_last_transaction'] = (last_transaction['ref_date'] - last_transaction['date']).dt.days

    features_list.append(last_transaction[['cust_id', 'days_since_last_transaction']])

    # Merge all aggregated features into final_features
    for feat_df in features_list:
        final_features = final_features.merge(feat_df, on='cust_id', how='left')

    # Handle categorical 'tenure_group' separately
    if 'tenure_group' in final_features.columns:
        # Ensure 'Bilinmiyor' is a valid category before filling
        if 'Bilinmiyor' not in final_features['tenure_group'].cat.categories:
            final_features['tenure_group'] = 'Bilinmiyor'
            final_features.loc[final_features['tenure'] <= 90, 'tenure_group'] = 'Çok Yeni (0-3 Ay)'
            final_features.loc[(final_features['tenure'] > 90) & (final_features['tenure'] <= 180), 'tenure_group'] = 'Yeni (3-6 Ay)'

    # Fill remaining numerical NaNs with 0
    numeric_cols = final_features.select_dtypes(include=np.number).columns
    final_features[numeric_cols] = final_features[numeric_cols].fillna(0)

    categorical_cols = final_features.select_dtypes(include=['object']).columns
    final_features[categorical_cols] = final_features[categorical_cols].fillna('Bilinmiyor')

    # Drop ref_date before returning
    if 'ref_date' in final_features.columns:
        final_features = final_features.drop(columns=['ref_date'])


    return final_features

ModuleNotFoundError: No module named 'autogluon'

## Eğitim ve Test Verilerini Oluşturmak

In [ ]:
full_train_data = create_features(customer_history, customers, referance_data, is_train=True)
test_data = create_features(customer_history, customers, referance_data_test, is_train=False, full_train_df_for_thresholds=full_train_data)


In [5]:
# ============================================================================
# 2. HASSAS ÖZELLİK SEÇİMİ
# ============================================================================
try:
    print("\nDaha önce eğitilmiş model yükleniyor...")
    model_path = 'ag_models'
    predictor = TabularPredictor.load(model_path)

    print("Özellik önem analizi yapılıyor...")
    feature_importance = predictor.feature_importance(data=full_train_data)

    # === DEĞİŞİKLİK BURADA ===
    # "Sıfıra çok yakın" için bir eşik değeri belirleyelim
    threshold = 0.0001 # Önem skoru 0.0001'den küçük olanlar

    # Katkısı çok az olan veya negatif olan özellikleri tespit et
    useless_features = feature_importance[feature_importance['importance'] < threshold].index.tolist()

    print("\n" + "="*60)
    print(f"Katkısı çok az olan (önem < {threshold}) veya negatif olan {len(useless_features)} adet özellik tespit edildi.")
    print(useless_features)
    print("="*60)

except Exception as e:
    print(f"\nModel yükleme veya özellik önemi analizi sırasında hata: {e}")
    useless_features = []


# ============================================================================
# 3. İŞE YARAMAYAN ÖZELLİKLERİ ÇIKARMA VE YENİDEN EĞİTİM
# ============================================================================
if useless_features:
    train_data_selected = full_train_data.drop(columns=useless_features, errors='ignore')
    test_data_selected = test_data.drop(columns=useless_features, errors='ignore')

    print(f"\n{len(useless_features)} özellik çıkarıldı. Yeni özellik sayısı: {len(train_data_selected.columns)}")



This means that the predictor was fit in an AutoGluon version `<=0.3.1`.



Daha önce eğitilmiş model yükleniyor...

Model yükleme veya özellik önemi analizi sırasında hata: [Errno 2] No such file or directory: '/content/ag_models/predictor.pkl'


## Yarışma Metrik Fonksiyonu

In [ ]:
def recall_at_k(y_true, y_prob, k=0.1): y_true, y_prob = np.asarray(y_true), np.asarray(y_prob); order = np.argsort(-y_prob, kind="mergesort"); top = order[:max(1, int(np.round(k * len(y_true))))]; return y_true[top].sum() / y_true.sum() if y_true.sum() > 0 else 0.0
def lift_at_k(y_true, y_prob, k=0.1): y_true, y_prob = np.asarray(y_true), np.asarray(y_prob); m = max(1, int(np.round(k * len(y_true)))); order = np.argsort(-y_prob, kind="mergesort"); precision_at_k = y_true[order[:m]].sum() / m; return precision_at_k / y_true.mean() if y_true.mean() > 0 else 0.0
def convert_auc_to_gini(auc): return 2 * auc - 1
def ing_hubs_datathon_metric(y_true, y_prob): weights = {"gini": 0.4, "recall_at_10perc": 0.3, "lift_at_10perc": 0.3}; baselines = {"roc_auc": 0.6925726757936908, "recall_at_10perc": 0.18469015795868773, "lift_at_10perc": 1.847159286784029}; roc_auc = max(0.5, roc_auc_score(y_true, y_prob)); scores = {"roc_auc": roc_auc, "recall_at_10perc": recall_at_k(y_true, y_prob), "lift_at_10perc": lift_at_k(y_true, y_prob)}; baselines["gini"] = convert_auc_to_gini(baselines["roc_auc"]); scores["gini"] = convert_auc_to_gini(scores["roc_auc"]); final_score = sum((scores[m] / baselines[m]) * weights[m] for m in weights); return final_score


## Autogluon

In [ ]:
train_data, val_data = train_test_split(full_train_data, test_size=0.2, random_state=42, stratify=full_train_data['churn'])

label = 'churn'
eval_metric = 'roc_auc'
predictor = TabularPredictor(
    label=label,
    eval_metric=eval_metric,
    path='ag_models' # Yeni bir klasöre kaydet
)

print(f"\nAutoGluon eğitimi {len(train_data)} satır ile başlıyor...")

# Modeli sizin belirttiğiniz ayarlar ile eğit
predictor.fit(
    train_data,
    time_limit=7200, # 30 dakika
    presets='best_quality',
    num_bag_folds=5,
    num_stack_levels=1,
    ag_args_fit={'num_gpus': 1}
)

## Sonuçların Gösterilmesi

In [ ]:
print("\n" + "="*60)
print("AUTOGLUON EĞİTİMİ TAMAMLANDI!")
print("="*60)
print("Model Performansları (Leaderboard):")
print(predictor.leaderboard(val_data, silent=False))
# Leaderboard, AutoGluon'un kendi içindeki validasyon skorlarını gösterir
leaderboard = predictor.leaderboard(train_data, silent=True)
print(leaderboard)
print("-" * 60)

# YENİ: Ayırdığımız doğrulama seti üzerinde tahmin yap
print("\nDoğrulama seti üzerinde yarışma metriği hesaplanıyor...")
y_val_pred = predictor.predict_proba(val_data, as_multiclass=False)
y_val_true = val_data[label]

# Yarışma metriği ile nihai skoru hesapla
final_competition_score = ing_hubs_datathon_metric(y_val_true, y_val_pred)

print(f"\nAUTOGLUON'UN NİHAİ YARIŞMA SKORU: {final_competition_score:.4f}")
print("="*60)

## SHAP

In [4]:
try:
    # Lütfen bir önceki adımda modeli kaydettiğiniz doğru 'path' adını buraya yazın
    model_path = 'ag_models'  # Bu ismi kontrol edin
    predictor = TabularPredictor.load(model_path)
    print(f"\n'{model_path}' yolundan model başarıyla yüklendi.")

    print("\nÖzellik önemi analizi başlıyor... (Bu işlem birkaç dakika sürebilir)")

    feature_importance = predictor.feature_importance(data=val_data)

    # Pandas'ın normalde satırları kısaltmasını engelle
    pd.set_option('display.max_rows', None)

    print("\n" + "="*60)
    print("MODELİN TÜM ÖZELLİKLERE VERDİĞİ ÖNEM DERECESİ (Büyükten Küçüğe):")
    print("="*60)
    print(feature_importance)

    # Ayarı normale döndür
    pd.reset_option('display.max_rows')

except Exception as e:
    print(f"\nModel yükleme veya analiz sırasında bir hata oluştu: {e}")
    print("Lütfen bir önceki adımda modeli başarıyla eğittiğinizden ve 'path' adının doğru olduğundan emin olun.")


Model yükleme veya analiz sırasında bir hata oluştu: name 'TabularPredictor' is not defined
Lütfen bir önceki adımda modeli başarıyla eğittiğinizden ve 'path' adının doğru olduğundan emin olun.


## Submission Dosyası Oluşturmak

In [8]:
print("\nEn iyi model ile submission dosyası oluşturuluyor...")
# AutoGluon, predict'i çağırdığınızda en iyi modeli otomatik olarak kullanır
y_test_pred = predictor.predict_proba(test_data, as_multiclass=False)

submission = pd.DataFrame({
    'cust_id': test_data['cust_id'],
    'churn': y_test_pred
})
submission.to_csv('submission_autogluon_full_data.csv', index=False)

print("\n'submission_autogluon_full_data.csv' dosyası başarıyla oluşturuldu.")



En iyi model ile submission dosyası oluşturuluyor...

'submission_autogluon_full_data.csv' dosyası başarıyla oluşturuldu.


## AUC

In [10]:
try:
    # Lütfen bir önceki adımda modeli kaydettiğiniz doğru 'path' adını buraya yazın
    model_path = 'ag_models'  # Bu ismi kontrol edin
    predictor = TabularPredictor.load(model_path)
    print(f"\n'{model_path}' yolundan model başarıyla yü yüklendi.")

    # Use transform_features on the full train data first
    full_train_data_transformed = predictor.transform_features(full_train_data)

    # Split the transformed data and the original data into train and validation sets
    train_indices, val_indices = train_test_split(full_train_data.index, test_size=0.2, random_state=42, stratify=full_train_data['churn'])

    train_data_transformed = full_train_data_transformed.loc[train_indices]
    val_data_transformed = full_train_data_transformed.loc[val_indices]

    y_train_true = full_train_data.loc[train_indices, 'churn']
    y_val_true = full_train_data.loc[val_indices, 'churn']


    # --- EĞITIM VERISI IÇIN SKOR HESAPLAMA ---
    print("\nEğitim verisi üzerinde skor hesaplanıyor...")
    y_train_pred = predictor.predict_proba(train_data_transformed.drop(columns=['churn'], errors='ignore'), as_multiclass=False) # Drop churn for prediction
    train_auc = roc_auc_score(y_train_true, y_train_pred)

    # --- DOĞRULAMA VERISI IÇIN SKOR HESAPLAMA ---
    print("Doğrulama verisi üzerinde skor hesaplanıyor...")
    y_val_pred = predictor.predict_proba(val_data_transformed.drop(columns=['churn'], errors='ignore'), as_multiclass=False) # Drop churn for prediction
    val_auc = roc_auc_score(y_val_true, y_val_pred)

    # --- SONUÇLARI KARŞILAŞTIRMA ---
    print("\n" + "="*60)
    print("            EZBERLEME (OVERFITTING) KONTROLÜ")
    print("="*60)
    print(f"  Eğitim Seti AUC Skoru      : {train_auc:.6f}")
    print(f"  Doğrulama Seti AUC Skoru   : {val_auc:.6f}")
    print("-" * 60)
    print(f"  Fark                       : {train_auc - val_auc:.6f}")
    print("="*60)

    if (train_auc - val_auc) > 0.05:
        print("\nSONUÇ: DİKKAT! Eğitim ve Doğrulama skorları arasında belirgin bir fark var.")
        print("Model, eğitim verisini bir miktar ezberlemiş olabilir (hafif overfitting).")
    else:
        print("\nSONUÇ: HARİKA! Eğitim ve Doğrulama skorları birbirine çok yakın.")
        print("Modeliniz, öğrendiği bilgiyi genelleştirme konusunda başarılı görünüyor.")

except Exception as e:
    print(f"\nModel yükleme veya analiz sırasında bir hata oluştu: {e}")


'ag_models' yolundan model başarıyla yü yüklendi.

Eğitim verisi üzerinde skor hesaplanıyor...
Doğrulama verisi üzerinde skor hesaplanıyor...

            EZBERLEME (OVERFITTING) KONTROLÜ
  Eğitim Seti AUC Skoru      : 0.744034
  Doğrulama Seti AUC Skoru   : 0.715718
------------------------------------------------------------
  Fark                       : 0.028316

SONUÇ: HARİKA! Eğitim ve Doğrulama skorları birbirine çok yakın.
Modeliniz, öğrendiği bilgiyi genelleştirme konusunda başarılı görünüyor.
